In [ ]:
import os
import pandas as pd
import numpy as np

# Função para ler os arquivos .s2p e retornar os dados
def ler_s2p(caminho_arquivo, s_param):
    frequencia = []
    s_real = []
    s_imag = []

    with open(caminho_arquivo, 'r') as f:
        for linha in f:
            # Ignorando comentários e linhas de cabeçalho
            if linha.startswith('#') or linha.startswith('!') or linha.strip() == '':
                continue

            # Cada linha contém: frequência, S11_real, S11_imag, S21_real, S21_imag, ...
            dados = linha.split()
            if len(dados) >= 9:  # Assegurando que temos todos os parâmetros (Frequência + 8 valores para S-params)
                frequencia.append(float(dados[0]))
               
                # Fazendo o match com o parâmetro S escolhido
                match s_param:
                    case 'S11':
                        i, j = 1, 2
                    case 'S21':
                        i, j = 3, 4
                    case 'S12':
                        i, j = 5, 6
                    case 'S22':
                        i, j = 7, 8 
                    case 'Smn':
                        raise ValueError("Parâmetro S inválido. Escolha entre S11, S21, S12 ou S22.")
            
                s_real.append(float(dados[i]))
                s_imag.append(float(dados[j]))

    return frequencia, s_real, s_imag

# Função para calcular o módulo de S em dB
def calcular_modulo_s_db(s_real):
    # Tratamento de erros para valores menores ou iguais a zero para evitar erros com log
    s_real = np.array(s_real)
    s_real[s_real <= 0] = np.finfo(float).eps  # Substitui valores <= 0 por um valor muito pequeno
   
    # Calculando o módulo em dB
    s_db = 10 * np.log(s_real)
   
    return s_db

# Função principal para ler todos os arquivos da pasta e salvar no Excel
def salvar_para_excel(caminho_pasta, nome_excel, s_param):
    arquivos = [f for f in os.listdir(caminho_pasta) if f.endswith('.s2p')]
   
    # DataFrames para armazenar os dados
    df_valores = pd.DataFrame()  # Para valores reais e imaginários
    df_valores_db = pd.DataFrame()  # Para valores em dB

    for arquivo in arquivos:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
       
        # Lendo o arquivo .s2p com o parâmetro S escolhido
        frequencia, s_real, s_imag = ler_s2p(caminho_arquivo, s_param)
       
        # Criando DataFrame para valores de Magnitue e Fase
        df_temp = pd.DataFrame({
            'Frequência (Hz)': frequencia,
            f'{arquivo}_{s_param}_real': s_real,
            f'{arquivo}_{s_param}_imag': s_imag
        })

        # Calculando valores em dB
        s_db = calcular_modulo_s_db(s_real)
       
        # Criando DataFrame para valores em dB
        df_temp_db = pd.DataFrame({
            'Frequência (Hz)': frequencia,
            f'{arquivo}_{s_param}_dB': s_db
        })

        # Juntando os dados ao DataFrame final (valores Mag e Pha)
        if df_valores.empty:
            df_valores = df_temp
            df_valores_db = df_temp_db
        else:
            df_valores = pd.merge(df_valores, df_temp, on='Frequência (Hz)', how='outer')
            df_valores_db = pd.merge(df_valores_db, df_temp_db, on='Frequência (Hz)', how='outer')

    # Salvando ambos os DataFrames em diferentes abas no Excel
    with pd.ExcelWriter(nome_excel, engine='openpyxl') as writer:
        df_valores.to_excel(writer, sheet_name='Valores Magnitude e Fase', index=False)
        df_valores_db.to_excel(writer, sheet_name='Valores em dB', index=False)

    print(f'Dados salvos com sucesso no arquivo {nome_excel}')

# Uso do script
caminho_pasta = '/caminho/diretório'  # Altere para o caminho da sua pasta de arquivos s2p
nome_excel = 'resultados_s2p.xlsx'
s_param = 'Smn'  # Escolha entre S11, S21, S12, S22
salvar_para_excel(caminho_pasta, nome_excel, s_param) 